# Multi-Species Multi-Reaction model

## Model Equations

## Example solving MSMR using PyBaMM

In [1]:
#%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import numpy as np

In [2]:
model = pybamm.lithium_ion.SPM(
    {
        "open-circuit potential": "MSMR",
        "particle": "MSMR",
    }
)

parameter_values = pybamm.ParameterValues("MSMR_Example")

In [3]:
# x_n
U_n = model.variables["Negative electrode open-circuit potential [V]"]
T = model.variables["Negative electrode temperature [K]"]
f = pybamm.constants.F / (pybamm.constants.R * T)
for i in range(6):
    U0 = pybamm.Parameter(f"U0_n_{i}")
    w = pybamm.Parameter(f"w_n_{i}")
    Xj = pybamm.Parameter(f"Xj_n_{i}")

    x_n = Xj / (1 + pybamm.exp(f * (U_n - U0) / w))
    model.variables[f"x{i}_n"] = x_n
    model.variables[f"X-averaged x{i}_n"] = pybamm.x_average(x_n)
    

# x_p
U_p = model.variables["Positive electrode open-circuit potential [V]"]
T = model.variables["Positive electrode temperature [K]"]
f = pybamm.constants.F / (pybamm.constants.R * T)
for i in range(4):
    U0 = pybamm.Parameter(f"U0_p_{i}")
    w = pybamm.Parameter(f"w_p_{i}")
    Xj = pybamm.Parameter(f"Xj_p_{i}")

    x_p = Xj / (1 + pybamm.exp(f * (U_p - U0) / w))
    model.variables[f"x{i}_p"] = x_p
    model.variables[f"X-averaged x{i}_p"] = pybamm.x_average(x_p)

In [4]:
experiment = pybamm.Experiment(
    [
        (
            "Discharge at 1C for 1 hour or until 3 V",
            "Rest for 1 hour",
            "Charge at C/3 until 4 V",
            "Hold at 4 V until 10 mA",
            "Rest for 1 hour",
        ),
    ]
)
sim = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment)
sim.solve()

In [5]:
sim.plot(
    [
        "X-averaged negative particle stoichiometry",
        "X-averaged positive particle stoichiometry",
        "X-averaged negative particle potential [V]",
        "X-averaged positive particle potential [V]",
        [f"X-averaged x{i}_n" for i in range(6)],
        [f"X-averaged x{i}_p" for i in range(4)],
        "X-averaged negative electrode open-circuit potential [V]",
        "X-averaged positive electrode open-circuit potential [V]",
        "Current [A]",
        "Voltage [V]",
    ]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=5.86646556905814, step=0.0586646556905814), …

## Comparison with ideal diffusion and "standard" OCV model

In [162]:
def U_n(sto):
    u_eq = (
        0.3635 * pybamm.exp(-439.3 * sto)
        + 0.6908
        + 0.5489 * pybamm.tanh(-30.07 * sto)
        - 0.0344 * pybamm.tanh(25.46 * (sto - 0.1713))
        - 0.0276 * pybamm.tanh(14.27 * (sto - 0.5270))

    )
    return u_eq

def U_p(sto):
    u_eq = (
        -0.3415 * sto
        + 4.116
        - 0.2835 * pybamm.tanh(4.181 * (sto - 0.2324))
        - 0.1020 * pybamm.tanh(29.61 * (sto - 1))
    )
    return u_eq

In [ ]:
parameter_values_2 = parameter_values.copy()
parameter_values_2.update({
    "Negative electrode OCP [V]": U_n,
    "Positive electrode OCP [V]": U_p,
})

## References

The relevant papers for this notebook are:

In [12]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Daniel R Baker and Mark W Verbrugge. Multi-species, multi-reaction model for porous intercalation electrodes: part i. model formulation and a perturbation solution for low-scan-rate, linear-sweep voltammetry of a spinel lithium manganese oxide electrode. Journal of The Electrochemical Society, 165(16):A3952, 2018.
[3] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[4] Scott G. Marquis, Valentin Sulzer, Robert Timms, Colin P. Please, and S. Jon Chapman. An asymptotic derivation of a single particle model w